# Conditional Execution

__!!!!!!! CAUTION! This example provides outdated information and is only left for reference purposes !!!!!!!__

Conditional exeuction is currently not supported (but we plan to get it back some time in the future).

__!!!!!!!!!!!!!!__

qupulse is desinged to support conditional execution of pulse (segments). This allows pulse designers to play back different waveforms depending on, e.g., environment data or state measurements of the quantum dot. Conditional execution may be implemented via trigger-based jumps to instructions directly on the playback device or emulated in software by choosing which pulse to send to the hardware for playback, if the hardware itself does not support branching.

Since the decision whether a condition will be evaluated software- or hardware-based depends on the hardware setup of the experiment and not on the pulse template, qupulse relies on an indirection scheme for conditions which is similar to the handling of parameters, as you will see in the following.

qupulse offers two `PulseTemplate` subclasses for conditional execution: `LoopPulseTemplate` and `BranchPulseTemplate`. 

`LoopPulseTemplate` takes an identifier for a condition and a subtemplate which is repeated as long as the condition evaluates to `True`. Let's assume that we want to construct a pulse that waits until some initialization is completed using a `TablePulseTemplate` representing a zero-pulse of length 5 and a `LoopPulseTemplate`:

In [1]:
from qupulse.pulses import TablePT
from qupulse.pulses.loop_pulse_template import WhileLoopPulseTemplate

wait_template = TablePT({0: [(5, 0)]})

loop_template = WhileLoopPulseTemplate('initialization_in_progress', wait_template)

`loop_template` is now configured to evaluate a condition called 'initialization_in_progress'. How this condition is implemented needs not to be specified to declare pulse templates.


We will now look into the actual implementation of conditions. The abstract interface of conditions is the `Condition` class. As mentioned in the beginning, conditions can be evaluated software- and hardware-based. The classes `SoftwareCondition` and `HardwareCondition` represent this in qupulse. Note that these classes don't do the actual evaluation but encapsulate it against the `Sequencer` and are used to generate correct sequences for both cases. Instances of `Condition`s are passed directly into the `Sequencer` and mapped to the `PulseTemplate`s via the identifier, similar to parameters.

## Software-Based Conditions
`SoftwareCondition` takes a callback function which is called to evaluate the condition (and thus must return a boolean value). During the sequencing process, this function will be called. If the return value is `True`, the subtemplate will be included in the instruction sequence and another evaluation will be made until the return value is `False`. The generated instruction sequence will thus contain a number of repetitions of the subtemplate but no actual jumping instructions, the loop is essentially unrolled. The callback function is passed an integer value indicating the current iteration of the evaluation.

As an example, we could use this to repeat the `wait_template` a fixed number of times, say 5, as follows:

In [2]:
from qupulse.pulses import Sequencer
from qupulse.pulses.conditions import SoftwareCondition

constant_repeat_condition = SoftwareCondition(lambda x: x < 5)
conditions = {'initialization_in_progress': constant_repeat_condition}

s = Sequencer()
parameters = {}
s.push(loop_template, parameters, conditions)
instructions = s.build()
print([instruction for instruction in instructions])

[<qupulse._program.instructions.MEASInstruction object at 0x00000000062329B0>, <qupulse._program.instructions.EXECInstruction object at 0x00000000074E8828>, <qupulse._program.instructions.MEASInstruction object at 0x0000000012B95CF8>, <qupulse._program.instructions.EXECInstruction object at 0x0000000013B78A58>, <qupulse._program.instructions.MEASInstruction object at 0x0000000013B78B70>, <qupulse._program.instructions.EXECInstruction object at 0x0000000013B78BA8>, <qupulse._program.instructions.MEASInstruction object at 0x0000000013B78C18>, <qupulse._program.instructions.EXECInstruction object at 0x0000000013B78C88>, <qupulse._program.instructions.MEASInstruction object at 0x0000000013B78CC0>, <qupulse._program.instructions.EXECInstruction object at 0x0000000013B78D30>, <qupulse._program.instructions.STOPInstruction object at 0x0000000013B78B38>]


We obtain an instruction sequence that repeats an execution instruction (for the `wait_template` waveform) five times. This is, of course, a very simple example. The callback function passed into the `SoftwareCondition` instance will more likely evaluate some measured data. Since this might not always be available in the sequencing pass, the callback may return `None`, which will interrupt the sequencing process similar to the [`requires_stop` method of the `Parameter` class](05Parameters.ipynb).

## Hardware-Based Conditions
Since software-based evaluation of conditions is slow and might interrupt the sequencing process, it might not always be applicable. If supported by the hardware, hardware-based condition evaluation is preferrable. For the sequencing process, this is represented by instances of `HardwareCondition`, which only take some identifier of the hardware trigger. This must first be obtained from the hardware *currently not implemented* and is embedded in the generated instruction sequence which will contain jump instructions.

Assuming we have a hardware setup with a trigger that fires continuously until temperature threshold is reached and we want our pulse from above to repeat as long as the trigger fires. We can realize this as follows:

In [3]:
from qupulse.pulses.conditions import HardwareCondition, Trigger

# stub representation for the trigger which must be obtained from the hardware in a real use case
temperature_trigger = Trigger()

temperature_trigger_condition = HardwareCondition(temperature_trigger)
conditions = {'initialization_in_progress': temperature_trigger_condition}

s = Sequencer()
parameters = {}
s.push(loop_template, parameters, conditions)
instructions = s.build()

# output the generated program
print("main instruction sequence:")
print([instruction for instruction in instructions])
print("\nlooped instruction sequence:")
print([instruction for instruction in instructions[0].target.block])

main instruction sequence:
[<qupulse._program.instructions.CJMPInstruction object at 0x0000000013B964A8>, <qupulse._program.instructions.STOPInstruction object at 0x00000000062329B0>]

looped instruction sequence:
[<qupulse._program.instructions.MEASInstruction object at 0x0000000013B962E8>, <qupulse._program.instructions.EXECInstruction object at 0x0000000013B96358>, <qupulse._program.instructions.GOTOInstruction object at 0x0000000013922A58>]


As you can see in the output, the sequencing process now procudes instructions that perform conditional jumps and returns with goto instructions. The details are omitted here, suffice it to say that the trigger is embedded in the conditional jump instruction and this sequence will loop on the hardware as long as the trigger fires.